# Connect a volume series to Class3D

This notebook connects each volume from a volume series (like those produced by 3D Var Display) to a 3D Class job. This saves the user the step of downloading a large volume series file, extracting it and uploading the volumes to the server, importing them, and manually connecting them to the job.

```{note}
Since this script unzips the volume series itself, it must be run on a system with access to CryoSPARC's filesystem.
```

In [1]:
from cryosparc.tools import CryoSPARC

cs = CryoSPARC(base_port=40000)
assert cs.test_connection()

Connection succeeded to CryoSPARC command_core at http://localhost:40002
Connection succeeded to CryoSPARC command_vis at http://localhost:40003
Connection succeeded to CryoSPARC command_rtp at http://localhost:40005


First, we find the correct job and determine the path to the zipped Volume Series.

In [4]:
project = cs.find_project("P296")
workspace = project.find_workspace("W2")
var3Ddisp_job = project.find_job("J45")

desired_volume_series = 0

series_path = var3Ddisp_job.load_output(
    f"series_{desired_volume_series}"
)["series/path"][0]
series_path = str(project.dir() / series_path)

In [5]:
unzip_path = series_path[:-4]
print(unzip_path)

import zipfile
with zipfile.ZipFile(series_path, "r") as z:
    z.extractall(unzip_path)

/bulk9/data/cryosparcdev_projects/rposert/CS-cb1-gpcr/J45/J45_component_000


Now we simply create and launch an Import Volumes job, pointing it at all of the `.mrc` files we just extracted.

In [7]:
lane = "cryoem9"

import_volumes_job = workspace.create_job(
    "import_volumes", 
    params={"volume_blob_path": unzip_path + "/*.mrc"}
)
import_volumes_job.queue(lane)
import_volumes_job.wait_for_done()

'completed'

All that is left now is to create a 3D classification job and plug in the particles. This allows the user to plug in a desired particle stack and launch the job from the CryoSPARC interface.

In [13]:
no_of_frames = len(import_volumes_job.doc['output_result_groups'])
vol_outputs = [(import_volumes_job.uid, f"imported_volume_{v}") for v in range(1, no_of_frames+1)]

new_class_3D_job = workspace.create_job(
    "class_3D", 
    connections={
        "volume": vol_outputs
    },
    params={
        "class3D_N_K": no_of_frames,
        "class3D_init_mode": "input"
    }
)